# Segmenting and Clustering Neighborhoods in Toronto 
#### Assignment completed by Megan Harrison
--------

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib
import json
import geocoder
from geopy.geocoders import Nominatim 
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from bs4 import BeautifulSoup

## 1. Obtain and Format Toronto Data from Wikipedia 

In [2]:
# Wikipedia URL where Toronto postal code table is located
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Extract the table from the webpage 
soup = BeautifulSoup((requests.get(url)).content,'lxml').table
table_rows = soup.find_all('tr')
table_rows = table_rows[1:]

# Format text into a dataframe
df_pc = pd.DataFrame()

for rows in table_rows:
    new_row = str(rows).split('<td>')

    new_row = pd.DataFrame([[new_row[1].replace('\n</td>\n',''),
                            new_row[2].replace('\n</td>\n',''),
                            new_row[3].replace('\n</td></tr>','')]])
    
    df_pc = pd.concat([new_row, df_pc])

In [3]:
# Format dataframe
df_pc = df_pc.rename(columns = {0:'PostalCode',1:'Borough',2:'Neighborhood'}).sort_values('PostalCode')
df_pc = df_pc[df_pc['Borough']!='Not assigned']
df_pc = df_pc.applymap(lambda x: str(x.replace(' /',',')))
df_pc = df_pc.reset_index().drop('index',axis=1)

In [4]:
df_pc.head(5)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
df_pc.shape

(103, 3)

## 2. Assign Coordinates to Postal Code Data

In [8]:
# Load coordinate data
df_coords = pd.read_csv('/Users/megan/Desktop/projects/Coursera_Capstone/Geospatial_Coordinates.csv')

# Merge data 
df_pc_coords = pd.merge(df_pc,df_coords, how = 'left', left_on = 'PostalCode', right_on = 'Postal Code')
df_pc_coords = df_pc_coords[['PostalCode','Borough','Neighborhood','Latitude','Longitude']]

df_pc_coords.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## 3. Cluster Toronto Neighborhoods

### Plot Neighborhoods

In [9]:
# Initiate map for plotting Toronto's neighborhoods
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent = 'ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [10]:
# Create a map of Toronto's neighborhoods
for lat, lng, borough, neighborhood in zip(df_pc_coords['Latitude'], df_pc_coords['Longitude'], df_pc_coords['Borough'], df_pc_coords['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Retrieve Foursquare Data on Toronto Neighborhoods

In [11]:
# Define Credentials, Version and Search Parameters
CLIENT_ID = '4TSDP1NHK4BCLHIIAKO4J13VN41KLOJSYQZN44BYFE4T34DR' 
CLIENT_SECRET = 'VOKSVM11BJK0AWEU2Y5NDVROOYUDOYAQBKDBH5XUJ4GX55QW'
VERSION = '20180605' 

radius = 500
LIMIT = 500

In [14]:
# Create function to explore venues in each neighborhood

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
# Run the function on each Toronto neighborhood
toronto_venues = getNearbyVenues(names=df_pc_coords['Neighborhood'],
                                 latitudes=df_pc_coords['Latitude'],
                                 longitudes=df_pc_coords['Longitude']
                                )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence P

In [16]:
print(toronto_venues.shape)
toronto_venues.head()

(2142, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [17]:
# Check to see how many venues were found in each neighborhood
toronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23


### Analyze Each Toronto Neighborhood

In [18]:
# One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# Move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[190]] + list(toronto_onehot.columns[0:190])+list(toronto_onehot.columns[191:269])
toronto_onehot = toronto_onehot[fixed_columns]

pd.set_option('display.max_columns', None)
toronto_onehot.head()

,Nail Salon,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Soccer Field,Social Club,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [19]:
# Group neighborhoods, take mean of frequency of occurence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

pd.set_option("max_rows", None)
toronto_grouped.head()

,Neighborhood,Nail Salon,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Soccer Field,Social Club,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [20]:
# Show each neighborhood with 5 most common venues
num_top_venues = 5

for neighborhood in toronto_grouped['Neighborhood']:
    print("----"+neighborhood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == neighborhood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0               Skating Rink  0.25
1  Latin American Restaurant  0.25
2             Breakfast Spot  0.25
3                     Lounge  0.25
4                 Nail Salon  0.00


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.22
1          Pool  0.11
2           Gym  0.11
3  Skating Rink  0.11
4      Pharmacy  0.11


----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0            Bank  0.10
1     Coffee Shop  0.10
2  Ice Cream Shop  0.05
3   Shopping Mall  0.05
4  Sandwich Place  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3  Japanese Restaurant  0.25
4           Nail Salon  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.09
1         Coffee Shop  0.09
2          Restaurant  0.09
3      Sandwich Place  0.09
4       Women's Store

4                 Bakery  0.05


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
               venue  freq
0  Mobile Phone Shop  0.25
1        Pizza Place  0.25
2     Sandwich Place  0.25
3           Bus Line  0.25
4        Men's Store  0.00


----Lawrence Manor, Lawrence Heights----
                    venue  freq
0          Clothing Store  0.20
1           Women's Store  0.13
2  Furniture / Home Store  0.13
3               Gift Shop  0.07
4             Event Space  0.07


----Lawrence Park----
               venue  freq
0               Park  0.33
1        Swim School  0.33
2           Bus Line  0.33
3      Metro Station  0.00
4  Mobile Phone Shop  0.00


----Leaside----
                    venue  freq
0             Coffee Shop  0.09
1     Sporting Goods Shop  0.09
2            Burger Joint  0.06
3                    Bank  0.06
4  Furniture / Home Store  0.06


----Little Portugal, Trinity----
                           venue  freq
0                   

4  Japanese Restaurant  0.06


----Victoria Village----
                   venue  freq
0            Coffee Shop   0.2
1      French Restaurant   0.2
2            Pizza Place   0.2
3           Hockey Arena   0.2
4  Portuguese Restaurant   0.2


----Westmount----
                venue  freq
0         Pizza Place  0.29
1      Discount Store  0.14
2  Chinese Restaurant  0.14
3      Sandwich Place  0.14
4         Coffee Shop  0.14


----Weston----
                        venue  freq
0                        Park   0.5
1           Convenience Store   0.5
2                 Men's Store   0.0
3  Modern European Restaurant   0.0
4           Mobile Phone Shop   0.0


----Wexford, Maryvale----
                       venue  freq
0                     Bakery  0.25
1                 Smoke Shop  0.25
2             Breakfast Spot  0.25
3  Middle Eastern Restaurant  0.25
4                 Nail Salon  0.00


----Willowdale----
              venue  freq
0       Pizza Place  0.08
1  Ramen Restaurant  0.08


In [21]:
# Create function to display top 10 venues for each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
# Create the new dataframe and display top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Pharmacy,Pool,Pub,Sandwich Place,Coffee Shop,Airport Terminal,College Stadium
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Gift Shop,Pizza Place,Sandwich Place,Bridal Shop,Diner,Restaurant,Deli / Bodega,Ice Cream Shop
3,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Women's Store
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Women's Store,Sushi Restaurant,Pharmacy,Pizza Place,Pub,Café


### Cluster Neighborhoods

In [23]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

In [24]:
# Add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_pc_coords

# Merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# Remove postal codes with no foursquare data
toronto_merged.dropna(subset=['Cluster Labels'],inplace=True)

In [25]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [26]:
print(' There are', len(toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]),'neighborhoods in cluster 0\n',
     'There are', len(toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]),' neighborhoods in cluster 1\n',
     'There are', len(toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]),' neighborhoods in cluster 2\n',
     'There are', len(toronto_merged.loc[toronto_merged['Cluster Labels'] == 3]),' neighborhoods in cluster 3\n',
     'There are', len(toronto_merged.loc[toronto_merged['Cluster Labels'] == 4]),' neighborhoods in cluster 4\n')

 There are 83 neighborhoods in cluster 0
 There are 12  neighborhoods in cluster 1
 There are 1  neighborhoods in cluster 2
 There are 2  neighborhoods in cluster 3
 There are 1  neighborhoods in cluster 4



#### *Given that the grand majority of observations have fallen into Cluster 0, I would like to take a deeper dive into this cluster, and create sub-clusters to gain more information*

### Sub Clustering of 'Cluster 0'

In [27]:
# get cluster 0 data
cluster_0 = toronto_merged.loc[toronto_merged['Cluster Labels']== 0]

# format to prepare to apply k-means clustering
cluster_0_grouped = pd.merge(cluster_0, toronto_grouped, on='Neighborhood')
cluster_0_grouped = cluster_0_grouped.iloc[:,17:]

In [28]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cluster_0_grouped)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([4, 0, 4, 1, 1, 4, 2, 4, 4, 4, 4, 1, 1, 1, 1, 4, 1, 1, 1, 1],
      dtype=int32)

In [29]:
# Add clustering labels
cluster_0.insert(0, 'Sub Cluster Labels', kmeans.labels_)

In [30]:
# create map
map_subclusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, subcluster in zip(cluster_0['Latitude'], cluster_0['Longitude'], cluster_0['Neighborhood'], cluster_0['Sub Cluster Labels']):
    label = folium.Popup(str(poi) + 'Sub Cluster ' + str(subcluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(subcluster)-1],
        fill=True,
        fill_color=rainbow[int(subcluster)-1],
        fill_opacity=0.7).add_to(map_subclusters)
       
map_subclusters

## Examine Clusters and Sub Clusters

### Cluster 0

In [31]:
print('There are',len(toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]),'neighborhoods in Cluster 0.')
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

There are 83 neighborhoods in Cluster 0.


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,0.0,Rental Car Location,Intersection,Bank,Medical Center,Mexican Restaurant,Breakfast Spot,Electronics Store,Dumpling Restaurant,Drugstore,Donut Shop
3,Scarborough,0.0,Coffee Shop,Indian Restaurant,Korean Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,Scarborough,0.0,Caribbean Restaurant,Fried Chicken Joint,Bank,Athletics & Sports,Thai Restaurant,Gas Station,Bakery,Hakka Restaurant,Drugstore,Donut Shop
5,Scarborough,0.0,Women's Store,Playground,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
6,Scarborough,0.0,Discount Store,Convenience Store,Coffee Shop,Train Station,Chinese Restaurant,Department Store,Hobby Shop,Women's Store,Dog Run,Dim Sum Restaurant
7,Scarborough,0.0,Bakery,Bus Line,Soccer Field,Ice Cream Shop,Bus Station,Intersection,Metro Station,Park,Donut Shop,Doner Restaurant
8,Scarborough,0.0,American Restaurant,Motel,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Women's Store
9,Scarborough,0.0,College Stadium,Skating Rink,General Entertainment,Café,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
10,Scarborough,0.0,Indian Restaurant,Vietnamese Restaurant,Pet Store,Furniture / Home Store,Chinese Restaurant,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant
11,Scarborough,0.0,Smoke Shop,Bakery,Breakfast Spot,Middle Eastern Restaurant,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Drugstore


### Sub Cluster 0.0

In [32]:
print('There is',len(cluster_0.loc[cluster_0['Sub Cluster Labels'] == 0]),'neighborhood in Sub Cluster 0.0.')
cluster_0.loc[cluster_0['Sub Cluster Labels'] == 0, cluster_0.columns[[2] + [6] + [0] + list(range(7, cluster_0.shape[1]))]]

There is 1 neighborhood in Sub Cluster 0.0.


,Borough,Cluster Labels,Sub Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,0.0,0,Coffee Shop,Indian Restaurant,Korean Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


### Sub Cluster 0.1

In [33]:
print('There are',len(cluster_0.loc[cluster_0['Sub Cluster Labels'] == 1]),'neighborhoods in Sub Cluster 0.1.')
cluster_0.loc[cluster_0['Sub Cluster Labels'] == 1, cluster_0.columns[[2] + [6] + [0] + list(range(7, cluster_0.shape[1]))]]

There are 60 neighborhoods in Sub Cluster 0.1.


,Borough,Cluster Labels,Sub Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,0.0,1,Women's Store,Playground,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
6,Scarborough,0.0,1,Discount Store,Convenience Store,Coffee Shop,Train Station,Chinese Restaurant,Department Store,Hobby Shop,Women's Store,Dog Run,Dim Sum Restaurant
13,Scarborough,0.0,1,Pharmacy,Pizza Place,Noodle House,Shopping Mall,Intersection,Chinese Restaurant,Fried Chicken Joint,Italian Restaurant,Thai Restaurant,Rental Car Location
15,Scarborough,0.0,1,Chinese Restaurant,Fast Food Restaurant,Nail Salon,Sandwich Place,Camera Store,Pizza Place,Supermarket,Pharmacy,Thrift / Vintage Store,Coffee Shop
17,North York,0.0,1,Dog Run,Mediterranean Restaurant,Golf Course,Pool,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Women's Store
18,North York,0.0,1,Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Tea Room,Bank,Bakery,Toy / Game Store,Food Court,Japanese Restaurant
22,North York,0.0,1,Coffee Shop,Pizza Place,Ramen Restaurant,Restaurant,Sandwich Place,Sushi Restaurant,Café,Grocery Store,Hotel,Ice Cream Shop
24,North York,0.0,1,Coffee Shop,Pizza Place,Ramen Restaurant,Restaurant,Sandwich Place,Sushi Restaurant,Café,Grocery Store,Hotel,Ice Cream Shop
26,North York,0.0,1,Gym,Coffee Shop,Restaurant,Asian Restaurant,Beer Store,Japanese Restaurant,Concert Hall,Italian Restaurant,Supermarket,Dim Sum Restaurant
27,North York,0.0,1,Gym,Coffee Shop,Restaurant,Asian Restaurant,Beer Store,Japanese Restaurant,Concert Hall,Italian Restaurant,Supermarket,Dim Sum Restaurant


### Sub Cluster 0.2

In [34]:
print('There is',len(cluster_0.loc[cluster_0['Sub Cluster Labels'] == 2]),'neighborhood in Sub Cluster 0.2.')
cluster_0.loc[cluster_0['Sub Cluster Labels'] == 2, cluster_0.columns[[2] + [6] + [0] + list(range(7, cluster_0.shape[1]))]]

There is 1 neighborhood in Sub Cluster 0.2.


,Borough,Cluster Labels,Sub Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,0.0,2,American Restaurant,Motel,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Women's Store


### Sub Cluster 0.3

In [35]:
print('There are',len(cluster_0.loc[cluster_0['Sub Cluster Labels'] == 3]),'neighborhoods in Sub Cluster 0.3.')
cluster_0.loc[cluster_0['Sub Cluster Labels'] == 3, cluster_0.columns[[2] + [6] + [0] + list(range(7, cluster_0.shape[1]))]]

There are 2 neighborhoods in Sub Cluster 0.3.


,Borough,Cluster Labels,Sub Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,0.0,3,Locksmith,Baseball Field,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
97,North York,0.0,3,Food Service,Baseball Field,Women's Store,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Falafel Restaurant


### Sub Cluster 0.4

In [36]:
print('There are',len(cluster_0.loc[cluster_0['Sub Cluster Labels'] == 4]),'neighborhoods in Sub Cluster 0.4.')
cluster_0.loc[cluster_0['Sub Cluster Labels'] == 4, cluster_0.columns[[2] + [6] + [0] + list(range(7, cluster_0.shape[1]))]]

There are 19 neighborhoods in Sub Cluster 0.4.


,Borough,Cluster Labels,Sub Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,0.0,4,Rental Car Location,Intersection,Bank,Medical Center,Mexican Restaurant,Breakfast Spot,Electronics Store,Dumpling Restaurant,Drugstore,Donut Shop
4,Scarborough,0.0,4,Caribbean Restaurant,Fried Chicken Joint,Bank,Athletics & Sports,Thai Restaurant,Gas Station,Bakery,Hakka Restaurant,Drugstore,Donut Shop
7,Scarborough,0.0,4,Bakery,Bus Line,Soccer Field,Ice Cream Shop,Bus Station,Intersection,Metro Station,Park,Donut Shop,Doner Restaurant
9,Scarborough,0.0,4,College Stadium,Skating Rink,General Entertainment,Café,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
10,Scarborough,0.0,4,Indian Restaurant,Vietnamese Restaurant,Pet Store,Furniture / Home Store,Chinese Restaurant,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant
11,Scarborough,0.0,4,Smoke Shop,Bakery,Breakfast Spot,Middle Eastern Restaurant,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Drugstore
12,Scarborough,0.0,4,Latin American Restaurant,Skating Rink,Lounge,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
19,North York,0.0,4,Japanese Restaurant,Café,Bank,Chinese Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Women's Store
30,North York,0.0,4,Grocery Store,Park,Baseball Field,Home Service,Airport,Food Truck,Gym / Fitness Center,Liquor Store,Bank,Shopping Mall
31,North York,0.0,4,Grocery Store,Park,Baseball Field,Home Service,Airport,Food Truck,Gym / Fitness Center,Liquor Store,Bank,Shopping Mall


### Cluster 1

In [37]:
print('There are',len(toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]),'neighborhoods in Cluster 1.')
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

There are 12 neighborhoods in Cluster 1.


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,1.0,Park,Coffee Shop,Playground,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
23,North York,1.0,Park,Bank,Convenience Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
25,North York,1.0,Park,Food & Drink Shop,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant,Event Space
40,East York,1.0,Park,Coffee Shop,Convenience Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Drugstore
44,Central Toronto,1.0,Park,Swim School,Bus Line,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant,Event Space
48,Central Toronto,1.0,Park,Playground,Trail,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Electronics Store,Doner Restaurant,Dance Studio
50,Downtown Toronto,1.0,Park,Playground,Trail,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Electronics Store,Doner Restaurant,Dance Studio
64,Central Toronto,1.0,Trail,Park,Sushi Restaurant,Jewelry Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
73,York,1.0,Park,Field,Hockey Arena,Trail,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Deli / Bodega
74,York,1.0,Park,Pool,Women's Store,Airport,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


### Cluster 2

In [38]:
print('There is',len(toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]),'neighborhood in Cluster 2.')
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

There is 1 neighborhood in Cluster 2.


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,North York,2.0,Martial Arts Dojo,Women's Store,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant


### Cluster 3

In [39]:
print('There are',len(toronto_merged.loc[toronto_merged['Cluster Labels'] == 3]),'neighborhoods in Cluster 3.')
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

There are 2 neighborhoods in Cluster 3.


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3.0,Construction & Landscaping,Bar,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
102,Etobicoke,3.0,Rental Car Location,Drugstore,Bar,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Women's Store


### Cluster 4

In [40]:
print('There is',len(toronto_merged.loc[toronto_merged['Cluster Labels'] == 4]),'neighborhood in Cluster 4.')
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

There is 1 neighborhood in Cluster 4.


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,4.0,Fast Food Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
